Notebook to compile and compare pixel value ranges of different manufacturers

In [57]:
import numpy as np
import h5py
import pandas as pd
from glob import glob
from tqdm import tqdm_notebook as tqdm

def pixel_metrics(fpath_h5):
    key = 'data_mask' if 'stanford' in fpath_h5 else 'data'
    scan_volume = np.array(h5py.File(fpath_h5)[key])
    contrasts = ['zero', 'low', 'full']
    metrics = {}
    for i, cont in enumerate(contrasts):
        metrics[cont] = (
            scan_volume[:, i, ...].max(),
            scan_volume[:, i, ...].mean()
        )
    
    return metrics

def compile_metrics(h5_base):
    print('Processing dir {}'.format(h5_base))
    metrics = []
    h5_files = [fpath_h5 for fpath_h5 in glob('{}/*.h5'.format(h5_base))]
        
    for fpath_h5 in tqdm(h5_files, total=len(h5_files)):
        metrics.append(pixel_metrics(fpath_h5))
    
    get_metric = lambda cont, m: [d[cont][m] for d in metrics]
    
    return {
        'Zero max': np.mean(get_metric('zero', 0)),
        'Zero mean': np.mean(get_metric('zero', 1)),
        'Low max': np.mean(get_metric('low', 0)),
        'Low mean': np.mean(get_metric('low', 1)),
        'Full max': np.mean(get_metric('full', 0)),
        'Full mean': np.mean(get_metric('full', 1))
    }

In [58]:
metrics = {
    'Tiantan (PHILIPS)': compile_metrics('/home/srivathsa/projects/studies/gad/tiantan_pp/data'),
    'Hoag (SIEMENS)': compile_metrics('/home/srivathsa/projects/studies/gad/hoag/preprocess/data'),
    'Stanford (GE)': compile_metrics('/home/srivathsa/projects/studies/gad/stanford/preprocess/data')
}

Processing dir /home/srivathsa/projects/studies/gad/tiantan_pp/data



Processing dir /home/srivathsa/projects/studies/gad/hoag/preprocess/data



Processing dir /home/srivathsa/projects/studies/gad/stanford/preprocess/data


In [59]:
df_metrics = pd.DataFrame(metrics)
df_metrics

,Hoag (SIEMENS),Stanford (GE),Tiantan (PHILIPS)
Full max,7.046328,4.753646,4.003297
Full mean,0.462270,0.186852,0.388875
Low max,6.014610,3.643961,4.181260
Low mean,0.350360,0.216201,0.402843
Zero max,6.181987,3.657233,4.354970
Zero mean,0.516113,0.219489,0.414779
